In [1]:
import random
import operator
from deap import creator, gp, base, tools, algorithms

In [2]:
class Card:
    def __init__(self, value, suit):
        self.value = value
        self.suit = suit
    
    def __str__(self):
        v = self.value
        if v == 14:
            v = 'A'
        elif v == 11:
            v = 'J'
        elif v == 12:
            v = 'Q'
        elif v == 13:
            v = 'K'
        return str(v) + self.suit

class Deck:
    def __init__(self):
        self.cards = []
        self.draw_queue = []
    
    def __str__(self):
        return_value = ""
        for card in self.draw_queue:
            return_value += str(card) + '\n'
        
        return return_value
    
    def setup(self, n=1):
        if len(self.cards) == 0:
            for i in range(0, n):
                for v in range(2, 15):
                    for s in ['S', 'D', 'C', 'H']:
                        self.cards.append(Card(v, s))

        self.draw_queue = []
        self.shuffle()
    
    def shuffle(self):
        self.draw_queue = list(self.cards)
        random.shuffle(self.draw_queue)

    def draw(self):
        return self.draw_queue.pop()

In [3]:
class Player:
    def __init__(self, hand, pot):
        self.hand = hand
        self.pot = pot
        self.folded = False
        self.current_bet = 0
        self.last_move = None
    
    def __str__(self):
        result = ''
        result += 'Hand: ' + str(self.hand[0]) + '  ' + str(self.hand[1])
        return result + '\nPot: ' + str(self.pot) + '\n'  
    
    def softReset(self):
        self.folded = False
        self.current_bet = 0
        self.last_move = None        
    
    def bid(self, amount):
        bid_value = amount
        if amount > self.pot:
            bid_value = self.pot
        
        self.pot = self.pot - bid_value
        self.current_bet += bid_value
        
        return bid_value

In [4]:
class Gameloop:
    def __init__(self, number_of_players, starting_player_pot, small_blind):
        self.number_of_players = number_of_players
        self.players = []
        for n in range(0, self.number_of_players):
            self.players.append(Player(0, starting_player_pot))
        self.current_player = 0
        self.button = 0
        self.small_blind = small_blind
        self.big_blind = 2 * small_blind
        self.deck = Deck()
        self.pot = 0
        self.board = []
        self.active_players = set()
        self.highest_bidder = None
        self.current_bid = 0
    
    def __str__(self):
        result = ''
        for player in self.players:
            result += str(player)
        
        return result
    
    def setup(self):
        self.deck.setup()
        self.pot = 0
        self.button = (self.button + 1) % self.number_of_players
        self.board = []
        
        small_blind_index = (self.button + 1) % self.number_of_players
        big_blind_index = (self.button + 2) % self.number_of_players
        
        #print str(small_blind_index) + ", " + str(big_blind_index)
        
        for i in range(0, self.number_of_players):
            hand = (self.deck.draw(), self.deck.draw())
            self.players[i].softReset()
            self.players[i].hand = hand
            if i == small_blind_index:
                self.pot += self.players[i].bid(self.small_blind)
            elif i == big_blind_index:
                self.pot += self.players[i].bid(self.big_blind)
        
        self.current_player = ((self.button + 3) % self.number_of_players)
        self.active_players = set(range(0, self.number_of_players))
        self.highest_bidder = big_blind_index
        self.current_bid = self.big_blind
        
        #self.button = (self.button + 1) % self.number_of_players
    
    def flop(self):
        self.board.append(self.deck.draw())
        self.board.append(self.deck.draw())
        self.board.append(self.deck.draw())
    
    def turn(self):
        self.board.append(self.deck.draw())
    
    def river(self):
        self.board.append(self.deck.draw())
    
    '''
    value  :   hand
    10 : Royal Straight Flush
    9  : Straight Flush
    8  : Four of a Kind
    7  : Full House
    6  : Flush
    5  : Straight
    4  : Three of a Kind
    3  : Two Pair
    2  : One Pair
    0  : High Card
    '''
    
    def flush(self, order_by_suit):
        max_value = 0
        max_suit = None
        for suit in order_by_suit:
            length = len(order_by_suit[suit])
            if length >= 5:
                max_value = length
                max_suit = suit
                break
        
        if max_value < 5:
            return (0, [])

        cards = order_by_suit[max_suit]
        
        if len(set([14, 13, 12, 11, 10]) & set(cards)) == 5: # Royal Straight Flush
            return (10, [])
        
        if 14 in cards:
            cards = [1] + cards # To guarantee that A's can also sequel with low card sequences
        difference_list = [cards[i+1]-cards[i] for i in range(0, len(cards)-1)]
        string_difference_list = ''.join([str(x) for x in difference_list])
        if '1111' in string_difference_list: # Straight Flush
            index = string_difference_list.find('1111')
            
            return (9, [cards[index+4]])
        
        return (6, [cards[-1], cards[-2], cards[-3], cards[-4], cards[-5]]) # Flush

    def xOfAKind(self, order_by_value, cards):
        values = order_by_value.values()
        keys = order_by_value.keys()
        card_set = set(cards)
        if 4 in values:
            card_value = keys[values.index(4)] 
            temp = card_set.difference(set([card_value]))
            return (8, [card_value, max(temp)]) # 4 of a Kind
        
        if (3 in values and 2 in values) or (3 in values and values.count(3) > 1):
            dict_of_keys = {2: [], 3: []}
            for key in order_by_value:
                if order_by_value[key] == 3:
                    dict_of_keys[3].append(key)
                    dict_of_keys[2].append(key)
                elif order_by_value[key] == 2:
                    dict_of_keys[2].append(key)
            
            max_3_card = max(dict_of_keys[3])
            max_2_card = max(set(dict_of_keys[2]).difference(set([max_3_card])))
            
            return (7, [max_3_card, max_2_card]) # Full House
        
        if 3 in values:
            dict_of_keys = {3: []}
            for key in order_by_value:
                if order_by_value[key] == 3:
                    dict_of_keys[3].append(key)
            
            card_value = max(dict_of_keys[3])
            max_1st_card_left = max(card_set.difference(set([card_value])))
            max_2nd_card_left = max(card_set.difference(set([card_value, max_1st_card_left])))
            
            return (4, [card_value, max_1st_card_left, max_2nd_card_left]) # 3 of a Kind
        
        if 2 in values:
            list_of_pair_keys = []
            for key in order_by_value:
                if order_by_value[key] == 2:
                    list_of_pair_keys.append(key)

            rank = 2
            if len(list_of_pair_keys) > 1:
                rank = 3
            
            max_1st_card = max(list_of_pair_keys)
            set_of_pair_keys = set(list_of_pair_keys).difference(set([max_1st_card]))
            max_2nd_card = 0
            if len(set_of_pair_keys) > 0:
                max_2nd_card = max(set_of_pair_keys)
            
            temp = list(card_set.difference(set([max_1st_card, max_2nd_card])))
            temp = sorted(temp)
            
            if rank == 3:
                return (3, [max_1st_card, max_2nd_card, temp[-1]])
            
            return (2, [max_1st_card, temp[-1], temp[-2], temp[-3], temp[-4]])
            #return 3 if values.count(2) >= 2 else 2 # Two or One Pair

        return (0, [])
    
    def straight(self, cards):
        if 14 in cards:
            cards = [1] + cards # To guarantee that A's can also sequel with low card sequences

        difference_list = [cards[i+1].value-cards[i].value for i in range(0, len(cards)-1)]
        string_difference_list = ''.join([str(x) for x in difference_list])
        if '1111' in string_difference_list: # Straight
            index = string_difference_list.find('1111')
            return (5, [cards[index+4].value])
    
        return (0, [])
    
    def rankHand(self, player_index):
        player = self.players[player_index]
        
        list_of_all_cards = self.board + [player.hand[0], player.hand[1]]
        list_of_all_cards.sort(key=operator.attrgetter('value'))
        
        order_by_suit = {'S': [], 'H': [], 'C': [], 'D': []}
        for card in list_of_all_cards:
            order_by_suit[card.suit].append(card.value)
        
        order_by_value = {}
        for card in list_of_all_cards:
            if card.value not in order_by_value:
                order_by_value[card.value] = 1
            else:
                order_by_value[card.value] += 1
        
        hand_rank = (0, [])
        
        hand_rank = self.flush(order_by_suit)
        if hand_rank[0] < 9:
            temp = self.xOfAKind(order_by_value, [card.value for card in list_of_all_cards])
            hand_rank = temp if temp > hand_rank else hand_rank
        if hand_rank[0] < 5:
            temp = self.straight(list_of_all_cards)
            hand_rank = temp if temp > hand_rank else hand_rank
        
        if hand_rank[0] == 0:
            temp = [x.value for x in player.hand]
            temp = sorted(temp)
            
            hand_rank = (0, [temp[-1], temp[-2]])
        
        return hand_rank
    
    def isBettingRoundOver(self):
        if len(self.active_players) <= 1:
            #print "1 player"
            return True
        
        if self.current_player == self.highest_bidder and self.players[self.current_player].last_move != None:
            #print "All checks"
            return True
        
        return False

    def nextPlayer(self):
        self.current_player = (self.current_player + 1) % self.number_of_players
        while self.current_player not in self.active_players:
            self.current_player = (self.current_player + 1) % self.number_of_players
    
    def raiseBet(self):
        player = self.players[self.current_player]
        
        amount = self.current_bid - player.current_bet + self.big_blind
        value_of_bid = player.bid(amount)
        self.pot += value_of_bid
        self.current_bid += self.big_blind
        self.highest_bidder = self.current_player

        player.last_move = 'raise'
        
        self.nextPlayer()
    
    def fold(self):
        player = self.players[self.current_player]
        self.active_players.remove(self.current_player)
        
        player.last_move = 'fold'
        
        self.nextPlayer()
    
    def check(self):
        player = self.players[self.current_player]
        
        amount = self.current_bid - player.current_bet
        if amount > 0:
            value_of_bid = player.bid(amount)
            self.pot += value_of_bid
            self.nextPlayer()
        
        player.last_move = 'check'
    
    def possible_moves(self):
        moves = ['raise', 'check', 'fold']
        
        #player = self.players[self.current_player]
        
        #if player.current_bet == self.current_bid:
        #    moves.append('check')
        
        return moves
    
    def executeMove(self, move):
        if move == 'raise':
            self.raiseBet()
        
        elif move == 'check':
            self.check()
        
        elif move == 'fold':
            self.fold()
    
    def isHandHigher(self, hand1, hand2):
        if hand1[0] > hand2[0]:
            return 0 # CHALLENGING HAND IS LOWER
        
        if hand2[0] > hand1[0]:
            return 1 # CHALLENGING HAND IS HIGHER
        
        for i in range(0, len(hand1[1])):
            if hand1[1][i] > hand2[1][i]:
                return 0 # CHALLENGING HAND IS LOWER
            if hand2[1][i] > hand1[1][i]:
                return 1 # CHALLENGING HAND IS HIGHER
        
        return 2 # TIE
    
    def winner(self):
        results = {}
        for player_index in self.active_players:
            results[player_index] = self.rankHand(player_index)
        
        #print results
        
        highest_hand = None
        winning_players = []
        
        for key in results:
            if highest_hand == None:
                highest_hand = results[key]
                winning_players.append(key)
            else:
                evaluation = self.isHandHigher(highest_hand, results[key])
                if evaluation == 1:
                    highest_hand = results[key]
                    winning_players = [key]
                elif evaluation == 2: 
                    winning_players.append(key)
        
        return winning_players
    

In [5]:
class GameHandler:
    def __init__(self, number_of_players, agents, starting_player_pot, small_blind):
        self.starting_player_pot = starting_player_pot
        self.game = Gameloop(number_of_players, starting_player_pot, small_blind)
        self.agents = agents
    
    def gameloop(self):
        for i in range(0, self.game.number_of_players):
            self.game.players[i].pot = self.starting_player_pot
            
        self.game.setup()
        
        for i in range(0, self.game.number_of_players):
            self.agents[i].setHand(self.game.players[i].hand)
            #self.game.players[i].pot = self.starting_player_pot
        
        counter = 0
        while not self.game.isBettingRoundOver():
            current_agent = self.game.current_player
            
            move = self.agents[current_agent].decide(self.game, self.game.possible_moves())
            self.game.executeMove(move)
            
            counter += 1
            if counter == 100:
                break
        
        self.game.flop()
        self.game.turn()
        self.game.river()
        
    
    def run(self):
        self.gameloop()
        
        winners = self.game.winner()
        num_of_winners = float(len(winners))
        for win_player_index in winners:
            self.game.players[win_player_index].pot += int(float(self.game.pot) / num_of_winners)        
        
        return winners, self.game.players[0].pot

    def runGetAllPlayerPots(self):
        self.gameloop()
        
        winners = self.game.winner()
        num_of_winners = float(len(winners))
        for win_player_index in winners:
            self.game.players[win_player_index].pot += int(float(self.game.pot) / num_of_winners)        
        
        return winners, [self.game.players[i].pot for i in range(self.game.number_of_players)]


In [6]:
import math

class TierHandAI:
    def __init__(self):
        pass
    
    def setHand(self, hand):
        self.hand = [hand[0].value, hand[1].value]
        self.isSameSuit = True if hand[0].suit == hand[1].suit else False
        self.hand_value = self.evaluateHand()
    
    def evaluateHand(self):
        hand_value = 0
        highest_card = max(self.hand)
        smallest_card = min(self.hand)
        
        if highest_card == 14:
            hand_value += 10.0
        elif highest_card == 13:
            hand_value += 8.0
        elif highest_card == 12:
            hand_value += 7.0
        elif highest_card == 11:
            hand_value += 6.0
        else:
            hand_value += float(highest_card) / 2.0
        
        if self.hand[0] == self.hand[1]:
            hand_value *= 2.0
            if self.hand[0] == 5:
                hand_value += 1.0
        
        if self.isSameSuit:
            hand_value += 2.0

        point_gap = highest_card - smallest_card
        if highest_card == 14 and smallest_card < 6:
            point_gap = smallest_card - 1

        if point_gap == 2:
            hand_value -= 1.0
        elif point_gap == 3:
            hand_value -= 2.0
        elif point_gap == 4:
            hand_value -= 4.0
        elif point_gap > 4:
            hand_value -= 5.0
        
        if point_gap < 2 and highest_card < 12:
            hand_value += 1.0
        
        if hand_value < 0:
            hand_value = 0.0

        return hand_value
        
    def decide(self, gamestate, available_actions):
        if self.hand_value == 0.0:
            return 'fold'
        
        MAX_BID_LEVEL = 5.0
        
        random_value = random.uniform(0, 1)
        current_total_bid_level = float(gamestate.current_bid) / float(gamestate.big_blind)
        #print(current_total_bid_level)
        
        normalized_hand_value = self.hand_value / 20.0
        
        #raise_chance = normalized_hand_value
        highest_bid_level = MAX_BID_LEVEL * normalized_hand_value
        #print(highest_bid_level)
        #fold_chance = (1.0 - normalized_hand_value) / 2.0
        fold_chance = (1.0 - math.log(self.hand_value, 20))
        
        if random_value < fold_chance:
            return 'fold'
        
        if highest_bid_level > current_total_bid_level:
            return 'raise'
        
        return 'check'
        
        #level_of_bid = float(gamestate.current_bid - gamestate.players[gamestate.current_player].current_bet)
        #level_of_bid = self.level_of_bid / float(gamestate.big_blind)
        
        # Hand Value interval = [0.0, 20.0]        

In [7]:
class AlwaysRaiseAI:
    def __init__(self):
        pass
    
    def setHand(self, hand):
        pass
    
    def decide(self, gamestate, available_actions):
        return 'raise'

In [8]:
from deap import creator, gp, base, tools, algorithms

import operator
import random as randomizer

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
#creator.create("Individual", list, fitness=creator.FitnessMax)
creator.create("GameHeuristicIndividualMax", gp.PrimitiveTree, fitness=creator.FitnessMax)

def IfThenElse(input_condition, output1, output2):
    if input_condition:
        return output1
    else:
        return output2

def isSameSuit(hand):
    if hand[0].suit == hand[1].suit:
        return True
    
    return False

def hasDoubles(hand):
    if hand[0].value == hand[1].value:
        return True
    
    return False

def hasCard(hand, value):
    if hand[0].value == value or hand[1].value == value:
        return True
    
    return False

def highestCard(hand):
    temp = [hand[0].value, hand[1].value]
    
    return max(temp)

def highestCardGE(hand, value):
    highest_card = highestCard(hand)
    
    if highest_card >= value:
        return True
    return False

def highestCardEQ(hand, value):
    highest_card = highestCard(hand)
    
    if highest_card == value:
        return True
    return False

def highestCardLE(hand, value):
    highest_card = highestCard(hand)
    
    if highest_card <= value:
        return True
    return False

def lowestCard(hand):
    temp = [hand[0].value, hand[1].value]
    
    return min(temp)

def lowestCardGE(hand, value):
    lowest_card = lowestCard(hand)
    
    if lowest_card >= value:
        return True
    return False

def lowestCardEQ(hand, value):
    lowest_card = lowestCard(hand)
    
    if lowest_card == value:
        return True
    return False

def lowestCardLE(hand, value):
    lowest_card = lowestCard(hand)
    
    if lowest_card <= value:
        return True
    return False

def eq(val1, val2):
    return operator.eq(val1, val2)

def ge(val1, val2):
    return operator.ge(val1, val2)

def le(val1, val2):
    return operator.le(val1, val2)

pset = gp.PrimitiveSetTyped("MoveSelector", [list, int], str)
#pset.renameArguments(ARG0="Hand", ARG1="PotSize")

pset.addTerminal(True, bool)
pset.addTerminal(False, bool)
for i in range(2,14):
    pset.addTerminal(i, int)
pset.addTerminal("raise", str)
pset.addTerminal("check", str)
pset.addTerminal("fold", str)
pset.addPrimitive(operator.and_, [bool, bool], bool)
#pset.addPrimitive(eq, [int, int], bool)
#pset.addPrimitive(ge, [int, int], bool)
#pset.addPrimitive(le, [int, int], bool)

pset.addPrimitive(IfThenElse, [bool, str, str], str)
pset.addPrimitive(isSameSuit, [list], bool)
pset.addPrimitive(hasDoubles, [list], bool)
pset.addPrimitive(hasCard, [list, int], bool)
pset.addPrimitive(highestCardGE, [list, int], bool)
pset.addPrimitive(highestCardEQ, [list, int], bool)
pset.addPrimitive(highestCardLE, [list, int], bool)
pset.addPrimitive(lowestCardGE, [list, int], bool)
pset.addPrimitive(lowestCardEQ, [list, int], bool)
pset.addPrimitive(lowestCardLE, [list, int], bool)


In [9]:
possible_strings = [
                "raise",
                "check",
                "fold"
               ]

possible_ints = range(2, 14)
#possible_bools_operations = ["and_", "eq", "ge", "le", "isSameSuit", "hasDoubles", "hasCard"]
#possible_bools_operations = ["and_", "isSameSuit", "hasDoubles", "hasCard", "highestCardGE", "highestCardEQ", "highestCardLE", "lowestCardGE", "lowestCardEQ", "lowestCardLE"]
possible_bools_operations = ["and_", "isSameSuit", "hasDoubles", "highestCardGE", "highestCardLE", "lowestCardGE", "lowestCardLE"]


In [10]:
import random
import multiprocessing

def generateBoolOp():
    operation = random.choice(possible_bools_operations)
    
    if operation == "and_":
        param1 = generateBoolOp()
        param2 = generateBoolOp()
    else:
        if len(operation) < 5:
            param1 = "ARG1"
            param2 = str(random.choice(possible_ints))
        else:
            param1 = "ARG0"
            if operation != "hasDoubles" and operation != 'isSameSuit':
                param2 = str(random.choice(possible_ints))
            else:
                return operation + "(" + param1 + ")"
            
    
    return operation + "(" + param1 + "," + param2 + ")"

def generateTerminal():
    term = random.choice(possible_strings)

    return term #+ "(Moves," + param1 + "," + param2 + ")"
    
def generateIndividualString(depth):
    if depth == 0:
        return generateTerminal()
    else:
        result = ""
        for i in range(0, depth):
            result += "IfThenElse(" + generateBoolOp() + "," + generateTerminal() + ","
        
        result += generateTerminal()
        for i in range(0, depth):
            result += ")"
        
        return result

def generateIndividual(depth, pset, type_=None):
    return gp.PrimitiveTree.from_string(generateIndividualString(depth), pset)

def parseAndCondition(condition):
    result = []
    i = 1
    while i < len(condition):
        if condition[i].name == "isSameSuit":
            result.append([isSameSuit])
            i += 1
        elif condition[i].name == "hasDoubles":
            result.append([hasDoubles])
            i += 1
        elif condition[i].name == "highestCardLE":
            result.append([highestCardLE, int(condition[i+2].name)])
        elif condition[i].name == "highestCardGE":
            result.append([highestCardGE, int(condition[i+2].name)])
        elif condition[i].name == "lowestCardLE":
            result.append([lowestCardLE, int(condition[i+2].name)])
        elif condition[i].name == "lowestCardGE":
            result.append([lowestCardGE, int(condition[i+2].name)])
        i += 1
    
    return result

def parseCondition(condition):
    #print [x.name for x in condition]
    result = []
    
    if condition[0].name == "isSameSuit":
        result = [isSameSuit]
    elif condition[0].name == "hasDoubles":
        result = [hasDoubles]
    elif condition[0].name == "highestCardLE":
        result = [highestCardLE, int(condition[-1].name)]
    elif condition[0].name == "highestCardGE":
        result = [highestCardGE, int(condition[-1].name)]
    elif condition[0].name == "lowestCardLE":
        result = [lowestCardLE, int(condition[-1].name)]
    elif condition[0].name == "lowestCardGE":
        result = [lowestCardGE, int(condition[-1].name)]
    elif condition[0].name == "and_":
        result = parseAndCondition(condition)
    
    return result

def parseAllConditions(conditions):
    result = []
    for condition in conditions:
        result.append(parseCondition(condition))
    
    return result

def handsThatMeetCondition(condition, hands):
    result = []
    
    if len(condition) == 1:
        for hand in hands:
            if condition[0](hand):
                result.append(hand)
    elif len(condition) == 2 and not isinstance(condition[0], list):
        for hand in hands:
            if condition[0](hand, condition[1]):
                result.append(hand)
    else:
        compiled_conditions = condition
        #print compiled_conditions
        for hand in hands:
            flag = True
            for cond in compiled_conditions:
                if len(cond) == 1:
                    if not cond[0](hand):
                        flag = False
                        break
                elif len(cond) == 2:
                    if not cond[0](hand, cond[1]):
                        flag = False
                        break

            if flag:
                result.append(hand)
    
    return result

def isTreeValid(tree, hands):
    conditions = []
    for i in range(0, len(tree)):
        if tree[i].name == "IfThenElse":
            subtree_slice = tree.searchSubtree(i)
            if tree[subtree_slice.stop-1].ret == str and tree[subtree_slice.stop-2].ret == str and tree[subtree_slice.stop-1].name == tree[subtree_slice.stop-2].name:
                return False
        
            subtree_slice = tree.searchSubtree(i+1)
            conditions.append(tree[subtree_slice.start:subtree_slice.stop])
        
    conditions = parseAllConditions(conditions)
    #print conditions
    current_hands = set(hands)
    for condition in conditions:
        temp = handsThatMeetCondition(condition, current_hands)
        
        #print "Start: " + str(len(current_hands))
        if len(temp) == len(current_hands) or len(temp) == 0:
            return False
        #print "End: " + str(len(temp))
        current_hands = current_hands.difference(set(temp))
    
    return True

all_hands = []
for i in range(2, 15):
    for j in range(i, 15):
        all_hands.append((Card(i, "D"), Card(j, "S")))
        if i != j:
            all_hands.append((Card(i, "D"), Card(j, "D")))

def isThereCopyOfIndividual(pop, ind):
    for p in pop:
        if primitiveTreeDistance(p, ind) == 0.0:
            return True
    return False

def makePopulation(n, individual_creation_function):
    population = []
    while len(population) < n:
        ind =  individual_creation_function()
        
        if not isThereCopyOfIndividual(population, ind) and isTreeValid(ind, all_hands):
            population.append(ind)
    
    return population

def individualForMutation(depth, pset, type_=None):
    result = ""
    if type_ == bool:
        result = generateBoolOp()
    if type_ == int:
        result = str(random.choice(possible_ints))
    if type_ == str:
        result = random.choice(possible_strings)

    return gp.PrimitiveTree.from_string(result, pset)

def makeMutation(indv, chosen, removed_elements):
    new_gene = toolbox.heuristic_mut(depth=1, pset=pset, type_=chosen.ret)
    #if chosen.ret != str and chosen.ret != bool and chosen.ret != int:
    while (chosen.name == new_gene[0].name):
        new_gene = toolbox.heuristic_mut(depth=1, pset=pset, type_=chosen.ret)

    index_for_mutation = indv.index(chosen)

    temp = []
    if chosen.name == "and_":
        temp_indv = gp.PrimitiveTree.from_string(str(gp.PrimitiveTree(indv)), pset)
        subtree_slice = temp_indv.searchSubtree(index_for_mutation)
        temp = indv[:index_for_mutation] + new_gene + indv[subtree_slice.stop:]
        for temp_element in indv[subtree_slice.start:subtree_slice.stop]:
            removed_elements.add(temp_element)
    elif isinstance(chosen, gp.Terminal):
        temp = indv[:index_for_mutation] + new_gene + indv[index_for_mutation+1:]
    elif isinstance(chosen, gp.Primitive):
        temp_indv = gp.PrimitiveTree.from_string(str(gp.PrimitiveTree(indv)), pset)
        subtree_slice = temp_indv.searchSubtree(index_for_mutation)
        temp = indv[:index_for_mutation] + new_gene + indv[subtree_slice.stop:]
        for temp_element in indv[subtree_slice.start:subtree_slice.stop]:
            removed_elements.add(temp_element)
    
    return list(temp)


def mutateExpressionTree(indv, mut_prob, pset):
    mutable_elements = set([])
    removed_elements = set([])
    
    for expr in indv:
        if expr.name != 'IfThenElse' and expr.name != "ARG0" and expr.name != "ARG1":
            mutable_elements.add(expr)
    
    #chosen = random.choice(mutable_elements)
    #current = gp
    count = 0
    for element in mutable_elements:
        if element not in removed_elements:
            removed_elements.add(element)
            n = random.uniform(0, 1)
            #print n
            if n <= mut_prob:
                count += 1
                chosen = element

                indv = makeMutation(indv, chosen, removed_elements)
    
    if count == 0:
        chosen = random.choice(list(mutable_elements))

        indv = makeMutation(indv, chosen, removed_elements)        
    
    return gp.PrimitiveTree.from_string(str(gp.PrimitiveTree(indv)), pset)

def mateIndividuals(ind1, ind2, depth, pset):
    r = random.randint(1,depth) # which level to crossover in
    n = random.randint(0,2) # which child to crossover
    
    index_of_ind1_tree = 0
    index_of_ind2_tree = 0
    
    counter = 1
    for i in range(0, len(ind1)):
        if ind1[i].arity == 3:
            if counter == r:
                index_of_ind1_tree = i
                break
            counter += 1

    counter = 1
    for i in range(0, len(ind2)):
        if ind2[i].arity == 3:
            if counter == r:
                index_of_ind2_tree = i
                break
            counter += 1
    
    subtree_slice_ind1 = ind1.searchSubtree(index_of_ind1_tree+1)
    for i in range(0, n):
        subtree_slice_ind1 = ind1.searchSubtree(subtree_slice_ind1.stop)

    subtree_slice_ind2 = ind2.searchSubtree(index_of_ind2_tree+1)
    for i in range(0, n):
        subtree_slice_ind2 = ind2.searchSubtree(subtree_slice_ind2.stop)

    new_ind1 = ind1[0:subtree_slice_ind1.start] + ind2[subtree_slice_ind2.start:subtree_slice_ind2.stop] + ind1[subtree_slice_ind1.stop:]
    new_ind2 = ind2[0:subtree_slice_ind2.start] + ind1[subtree_slice_ind1.start:subtree_slice_ind1.stop] + ind2[subtree_slice_ind2.stop:]

    return (gp.PrimitiveTree.from_string(str(gp.PrimitiveTree(new_ind1)), pset), gp.PrimitiveTree.from_string(str(gp.PrimitiveTree(new_ind2)), pset))

tree_depth = 2

toolbox = base.Toolbox()
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("heuristic", generateIndividual, depth=tree_depth, pset=pset)
toolbox.register("individual", tools.initIterate, creator.GameHeuristicIndividualMax, toolbox.heuristic)
#toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("population", makePopulation, individual_creation_function=toolbox.individual)
toolbox.register("select", tools.selBest)
#toolbox.register("select", tools.selTournament, tournsize=3)
#toolbox.register("select", tools.selDoubleTournament, parsimony_size=1.7, fitness_first=True, fitness_size=3)
toolbox.register("mate", mateIndividuals, depth=tree_depth, pset=pset)
#toolbox.register("mate", gp.cxOnePoint)
toolbox.register("heuristic_mut", individualForMutation, depth=tree_depth)
#toolbox.register("mutate", gp.mutUniform, expr=toolbox.heuristic_mut, pset=pset)
toolbox.register("mutate", mutateExpressionTree, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=tree_depth)) #Depth to allow ANDs
#toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=tree_depth+2)) #Depth + 2 to allow ANDs
#toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=3)) #Depth + 2 to allow ANDs

#pool = multiprocessing.Pool()
#toolbox.register("map", pool.map)

#pop = toolbox.population(n=50)
#hof = tools.HallOfFame(3)
#pop, log = algorithms.eaMuPlusLambda(pop, toolbox, mu=10*k, 
#                                     lambda_=30*k, cxpb=.5, mutpb=.1, ngen=ngen, 
#                                     stats=mstats, halloffame=hof, verbose=True)

#for i, item in enumerate(hof.items):
#    print((i,str(item)))

#print generateIndividualString(1)
#print (generateIndividual(1, pset))
#ind1 = toolbox.individual()
#ind2 = toolbox.individual()
#print(ind1)
#print(ind2)
#print(ind1.height)
#print(ind2.height)

#mut = toolbox.mutate(ind1)
#print(mut)
#for i in mut:
#    print(i)
#    print(i.height)


In [11]:
class HeuristicAI:
    def __init__(self):
        pass
    
    def setHeuristic(self, heuristic):
        self.heuristic = heuristic
    
    def setHand(self, hand):
        self.hand = hand
    
    def decide(self, gamestate, available_actions):
        #hand = self.gamestate.players[self.gamestate.current_player]
        pot_size = int(float(gamestate.current_bid) / float(gamestate.big_blind))
        
        return self.heuristic(self.hand, pot_size)

In [12]:
BIG_BLIND = 10
TOTAL_PLAYER_POT = 1000
NUM_OF_PLAYERS = 8

def runSimulation(individual, n):
    number_of_players = NUM_OF_PLAYERS
    total_player_pot = TOTAL_PLAYER_POT
    small_blind = BIG_BLIND / 2

    #agents = [HeuristicAI(), HeuristicAI()]
    #for i in range(2, number_of_players):
    agents = [HeuristicAI()]
    for i in range(1, number_of_players-1):
        agents.append(TierHandAI())
    agents.append(AlwaysRaiseAI())

    agents[0].setHeuristic(toolbox.compile(individual))
    #agents[1].setHeuristic(toolbox.compile(individual))
    
    gh = GameHandler(number_of_players, agents, total_player_pot, small_blind)
    
    results = []
    agent_pot = []
    
    for i in range(0, n):
        result, money = gh.run()
        results.append(result)
        agent_pot.append(money)
    
    return (results, agent_pot)

def evaluate(player, n=8000):
    results, money = runSimulation(player, n)
    score = {'wins': 0.0, 'ties': 0.0, 'losses': 0.0}
    money_lost = 0.0
    
    '''
    for i in range(0, len(results)):
    #for result in results:
        result = results[i]
        current_pot = money[i]
        if 0 in result:
            if len(result) == 1:
                score['wins'] += 1
            else:
                score['ties'] += 1
        else:
            score['losses'] += 1
            money_lost += float(TOTAL_PLAYER_POT - current_pot) / float(BIG_BLIND)
            #print money_lost

    #penalty = 1.0 if score['losses'] != n else 100.0
    #win_ratio = float(score['wins']) / float(n)
    #penalty *= 100.0 if win_ratio < 0.15 else 1.0
    
    return (score['wins'] + (score['ties'] / 2.0) - (penalty * (money_lost / 1.0)), )
    '''
    return (sum(money),)

#print evaluate(toolbox.individual())

In [13]:
import random, itertools
import multiprocessing as mp

def primitiveTreeDistance(ind1, ind2):
    result = 0.0
    
    min_length = min(len(ind1), len(ind2))
    factor = 1.0 / float(min_length)
    for i in range(0, min_length):
        if ind1[i].name != ind2[i].name:
            result += factor

    if len(ind1) != len(ind2):
        result += factor
        
    return result if result <= 1.0 else 1.0

def findFitness(population):
    pop = population
    
    pool = mp.Pool(mp.cpu_count())
    fitness_list = pool.map(toolbox.evaluate, pop)
    pool.terminate()
    
    for i in range(0, len(fitness_list)):
        #pop[i].fitness = creator.FitnessMax
        #pop[i].fitness.values = fitness_list[i]
        pop[i].fitness = fitness_list[i]
    
    return pop

def readjustFitnessByDistance(population):
    pop = population
    best = toolbox.select(pop, 1)[0]
    weight = 10000.0
    
    for ind in pop:
        if ind != best:
            dist = 1.0 - primitiveTreeDistance(best, ind)
            ind.fitness = (ind.fitness[0] - (weight * dist), )

def mateBest(best):
    new_indv = []
    
    for i in range(0, len(best)-1):
        for j in range(i+1, len(best)):
            children = toolbox.mate(gp.PrimitiveTree.from_string(str(best[i]), pset), gp.PrimitiveTree.from_string(str(best[j]), pset))
            new_indv.extend(children)

    return new_indv

def mateElite(elite, size_of_children=60):
    children = []
    
    pairings = tuple(itertools.combinations(elite, 2))
    selected = random.sample(pairings, int(float(size_of_children) / 2.0))
    
    for i in range(0, len(selected)):
        children.extend(toolbox.mate(gp.PrimitiveTree.from_string(str(selected[i][0]), pset), gp.PrimitiveTree.from_string(str(selected[i][1]), pset)))

    return children
        
def mutateChildren(children, mut_prob, pop):
    mut_children = []
    
    for child in children:
        #rand_value = random.uniform(0.0, 1.0)
        #if rand_value <= mut_prob:
        #child = toolbox.mutate(child, mut_prob)

        while isThereCopyOfIndividual(pop + mut_children, child) or not isTreeValid(child, all_hands):
            child = toolbox.mutate(child, mut_prob, pset=pset)
        
        mut_children.append(child)

    return mut_children

def createMutationIndv(indv, mut_prob, pop):
    mutation = gp.PrimitiveTree.from_string(str(indv), pset)
    mut_indv = toolbox.mutate(mutation, mut_prob, pset=pset)
    
    while isThereCopyOfIndividual(pop, mut_indv) or not isTreeValid(mut_indv, all_hands):
        mut_indv = toolbox.mutate(mutation, mut_prob, pset=pset)
    
    return mut_indv

def generation(population, mut_prob, children_mut_prob, k):
    pop = population
    pop_size = len(pop)

    elite_size = int(0.5 * pop_size)
    elite = toolbox.select(pop, elite_size)
    elite_mutations = []
    for i in range(0, int(len(elite) * 0.4)):
        elite_mutations.append(createMutationIndv(elite[i], mut_prob, elite + elite_mutations))

    children = mateElite(elite, int(0.3 * pop_size))
    children = mutateChildren(children, children_mut_prob, elite + elite_mutations)

    pop = elite + elite_mutations + children
    pop = findFitness(pop)
    
    return pop

def gen0(population):
    pop = population
    pop = findFitness(pop)
    #readjustFitnessByDistance(pop)
    
    return pop

def createLogFile(filename, start_string):
    f = open(filename, 'w')
    f.write(start_string)
    f.close()

def logToFile(filename, input_string):
    f = open(filename, 'a')
    f.write(input_string)
    f.close()

In [ ]:
import time

toolbox.register("evaluate", evaluate, n=80000)

filename = "gen-output.txt"

ngen=30
k=10
mut_prob = 0.3
children_mut_prob = 0.1
pop = toolbox.population(n=200)
#hof = tools.HallOfFame(3)

start = time.time()
createLogFile(filename, "Gen 0 Start\n")

pop = gen0(pop)

logToFile(filename, "Gen 0 End\nEllapsed Time: " + str(time.time() - start) + "\n")

for i in range(0, ngen):
    start = time.time()
    logToFile(filename, "Gen " + str(i+1) + "\n")
    
    pop = generation(pop, mut_prob, children_mut_prob, k)
    
    top = toolbox.select(pop, 3)
    logToFile(filename, "Fitness: " + str(top[0].fitness) + "\n")
    logToFile(filename, "Top: " + str(top[0]) + "\n")
    logToFile(filename, "Fitness: " + str(top[1].fitness) + "\n")
    logToFile(filename, "Top: " + str(top[1]) + "\n")
    logToFile(filename, "Fitnees: " + str(top[2].fitness) + "\n")
    logToFile(filename, "Top: " + str(top[2]) + "\n")
    
    logToFile(filename, "Ellapsed Time: " + str(time.time() - start) + "\n")
    

print("==================================")
print("==================================")
print("==================================")

best = toolbox.select(pop, 3)
for b in best:
    logToFile(filename, str(b) + "\n")
    logToFile(filename, str(b.fitness) + "\n")

In [14]:
def generateAllHeuristicTrees(depth=1):
    main = "IfThenElse("
    one_parameter_functions = ["isSameSuit", "hasDoubles"]
    two_parameter_functions = ["highestCardGE", "highestCardLE", "lowestCardGE", "lowestCardLE"]
    int_parameters = range(2, 14)
    string_parameters = ["'raise'", "'check'", "'fold'"]
    special_operations = ["and_"]

    first_param = []
    
    for func in one_parameter_functions:
        result = func + '(ARG0)'
        first_param.append(result)
    
    for func in two_parameter_functions:
        for param_int in int_parameters:
            result = func + '(ARG0,'
            result += str(param_int) + ')'
            first_param.append(result)
    
    '''
    temp = []
    for func1 in first_param:
        for func2 in first_param:
            if func1.split("(")[0] != func2.split("(")[0]:
                temp.append("and_(" + func1 + "," + func2 + ")")
    
    first_param.extend(temp)
    '''
                
    second_param = list(string_parameters)
    
    third_param = []
    if depth == 1:
        third_param = list(string_parameters)
    else:
        third_param = generateAllHeuristicTrees(depth-1)
    
    trees = []
    for x in first_param:
        for y in second_param:
            for z in third_param:
                trees.append(main + x + ',' + y + ',' + z + ")")
    
    return trees

def generateListPrimitiveTreesFromListStrings(list_strings):
    result = []
    for tree_string in list_strings:
        result.append(gp.PrimitiveTree.from_string(tree_string, pset))
    
    return result

In [ ]:
## Exhaustive search
toolbox.register("evaluate", evaluate, n=160000)

pop = generateListPrimitiveTreesFromListStrings(generateAllHeuristicTrees(1))
pop = findFitness(pop)

for x in toolbox.select(pop, 3):
    print x
    print "Fitness: " + str(x.fitness)

In [16]:
pop = generateListPrimitiveTreesFromListStrings(generateAllHeuristicTrees(1))
print len(pop)
for p in pop:
    if not isTreeValid(p, all_hands):
        pop.remove(p)

print len(pop)

toolbox.register("evaluate", evaluate, n=160000)
pop = findFitness(pop)

for x in toolbox.select(pop, 10):
    print x
    print "Fitness: " + str(x.fitness)

450
343
IfThenElse(highestCardLE(ARG0, 9), 'fold', 'raise')
Fitness: (178686756,)
IfThenElse(highestCardGE(ARG0, 9), 'raise', 'fold')
Fitness: (178654904,)
IfThenElse(highestCardLE(ARG0, 8), 'fold', 'raise')
Fitness: (178622545,)
IfThenElse(highestCardLE(ARG0, 9), 'check', 'raise')
Fitness: (178460030,)
IfThenElse(highestCardLE(ARG0, 8), 'check', 'raise')
Fitness: (178315083,)
IfThenElse(highestCardGE(ARG0, 10), 'raise', 'fold')
Fitness: (178278805,)
IfThenElse(highestCardGE(ARG0, 11), 'raise', 'fold')
Fitness: (178247703,)
IfThenElse(highestCardLE(ARG0, 7), 'fold', 'raise')
Fitness: (178199094,)
IfThenElse(highestCardLE(ARG0, 7), 'check', 'raise')
Fitness: (178155341,)
IfThenElse(highestCardGE(ARG0, 10), 'raise', 'check')
Fitness: (178140606,)


In [494]:
for i in range(0,1000):
    ind = toolbox.individual()
    try:
        test = toolbox.mutate(ind, pset=pset)
    except:
        print ind
#print ind
#print toolbox.mutate(ind, pset=pset)

IfThenElse(and_(hasCard(ARG0, 11), highestCardEQ(ARG0, 8)), 'fold', IfThenElse(and_(highestCardLE(ARG0, 8), lowestCardEQ(ARG0, 9)), 'raise', 'fold'))
IfThenElse(and_(hasCard(ARG0, 2), isSameSuit(ARG0)), 'raise', IfThenElse(highestCardLE(ARG0, 12), 'fold', 'fold'))
IfThenElse(and_(highestCardGE(ARG0, 7), isSameSuit(ARG0)), 'check', IfThenElse(lowestCardGE(ARG0, 2), 'check', 'fold'))
IfThenElse(and_(isSameSuit(ARG0), isSameSuit(ARG0)), 'fold', IfThenElse(isSameSuit(ARG0), 'fold', 'raise'))


In [508]:
ind = gp.PrimitiveTree.from_string("IfThenElse(and_(isSameSuit(ARG0), isSameSuit(ARG0)), 'fold', IfThenElse(isSameSuit(ARG0), 'fold', 'raise'))", pset)
print toolbox.mutate(ind, pset=pset)

Chosen: isSameSuit
New Gene:hasCard(ARG0, 11)
Index: 2
IfThenElse(and_(hasCard(ARG0, 11), isSameSuit(ARG0)), 'fold', IfThenElse(isSameSuit(ARG0), 'fold', 'raise'))


In [21]:
count = 0
for j in range(0, 10000):
    o = primitiveTreeDistance(toolbox.individual(), toolbox.individual())
    if o > 1.0:
        count += 1

print count

0


In [59]:
ind1 = toolbox.individual()
ind2 = toolbox.individual()
print ind1
print ind2
temp = toolbox.mate(ind1, ind2)
print temp[0]
print temp[1]

IfThenElse(highestCardGE(ARG0, 13), 'check', IfThenElse(lowestCardEQ(ARG0, 3), 'check', 'fold'))
IfThenElse(lowestCardGE(ARG0, 2), 'fold', IfThenElse(lowestCardGE(ARG0, 13), 'check', 'fold'))
IfThenElse(highestCardGE(ARG0, 13), 'check', 'check')
IfThenElse(highestCardGE(ARG0, 13), 'check', IfThenElse(lowestCardEQ(ARG0, 3), 'check', 'fold'))


In [134]:
def mateIndividualsOld(ind1, ind2, depth):
    r = random.randint(1,depth) # which level to crossover in
    n = random.randint(0,2) # which child to crossover
    
    index_of_ind1_tree = 0
    index_of_ind2_tree = 0
    
    counter = 1
    for i in range(0, len(ind1)):
        if ind1[i].arity == 3:
            if counter == r:
                index_of_ind1_tree = i
                break
            counter += 1

    counter = 1
    for i in range(0, len(ind2)):
        if ind2[i].arity == 3:
            if counter == r:
                index_of_ind2_tree = i
                break
            counter += 1
    
    #print index_of_ind1_tree
    #print index_of_ind2_tree
    
    subtree_slice_ind1 = ind1.searchSubtree(index_of_ind1_tree+1)
    for i in range(0, n):
        subtree_slice_ind1 = ind1.searchSubtree(subtree_slice_ind1.stop)

    subtree_slice_ind2 = ind2.searchSubtree(index_of_ind2_tree+1)
    for i in range(0, n):
        subtree_slice_ind2 = ind2.searchSubtree(subtree_slice_ind2.stop)

    #print subtree_slice_ind1
    #print ind1[subtree_slice_ind1.start:subtree_slice_ind1.stop]

    #print subtree_slice_ind2
    #print ind2[subtree_slice_ind2.start:subtree_slice_ind2.stop]
    new_ind1 = ind1[0:subtree_slice_ind1.start] + ind2[subtree_slice_ind2.start:subtree_slice_ind2.stop] + ind1[subtree_slice_ind1.stop:]
    new_ind2 = ind2[0:subtree_slice_ind2.start] + ind1[subtree_slice_ind1.start:subtree_slice_ind1.stop] + ind2[subtree_slice_ind2.stop:]

    return (new_ind1, new_ind2)


In [21]:
ind = gp.PrimitiveTree.from_string("IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(highestCardGE(ARG0, 10), 'check', 'fold'))", pset)
ind2 = gp.PrimitiveTree.from_string("IfThenElse(hasDoubles(ARG0), 'raise', 'check')", pset)
ind3 = gp.PrimitiveTree.from_string("IfThenElse(highestCardGE(ARG0,2), 'fold', 'fold')", pset)
ind4 = gp.PrimitiveTree.from_string("IfThenElse(lowestCardLE(ARG0, 13), 'fold', IfThenElse(hasDoubles(ARG0), 'check', IfThenElse(hasDoubles(ARG0), 'fold', 'fold')))", pset)
ind5 = gp.PrimitiveTree.from_string("IfThenElse(hasCard(ARG0, 6), 'fold', IfThenElse(lowestCardGE(ARG0, 6), 'check', IfThenElse(hasDoubles(ARG0), 'fold', 'fold')))", pset)
print evaluate(ind, n=160000)
print evaluate(ind2, n=160000)
#print evaluate(ind3, n=80000)
#print evaluate(ind4, n=80000)
#print evaluate(ind5, n=80000)


(164682897,)
(164395889,)


In [33]:
ind_exhaustive = gp.PrimitiveTree.from_string("IfThenElse(highestCardGE(ARG0, 10), 'raise', 'check')", pset)
ind_size_1 = gp.PrimitiveTree.from_string("IfThenElse(highestCardGE(ARG0, 9), 'raise', 'fold')", pset)
ind_size_2 = gp.PrimitiveTree.from_string("IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(highestCardGE(ARG0, 10), 'raise', 'fold'))", pset)
ind_size_3 = gp.PrimitiveTree.from_string("IfThenElse(highestCardGE(ARG0, 10), 'raise', IfThenElse(lowestCardLE(ARG0, 6), 'fold', IfThenElse(lowestCardGE(ARG0, 4), 'raise', 'check')))", pset)
ind_size_4 = gp.PrimitiveTree.from_string("IfThenElse(highestCardGE(ARG0, 11), 'raise', IfThenElse(lowestCardGE(ARG0, 7), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(hasDoubles(ARG0), 'fold', 'check'))))", pset)
ind_size_5 = gp.PrimitiveTree.from_string("IfThenElse(and_(highestCardLE(ARG0, 3), highestCardGE(ARG0, 13)), 'fold', IfThenElse(highestCardGE(ARG0, 10), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(isSameSuit(ARG0), 'check', IfThenElse(isSameSuit(ARG0), 'raise', 'fold')))))", pset)

exhaustive_eval = evaluate(ind_exhaustive, n=160000)
gen_size_1_eval = evaluate(ind_size_1, n=160000)
gen_size_2_eval = evaluate(ind_size_2, n=160000)
gen_size_3_eval = evaluate(ind_size_3, n=160000)
gen_size_4_eval = evaluate(ind_size_4, n=160000)
gen_size_5_eval = evaluate(ind_size_5, n=160000)

In [36]:
print exhaustive_eval[0]
print gen_size_1_eval[0]
print gen_size_2_eval[0]
print gen_size_3_eval[0]
print gen_size_4_eval[0]
print gen_size_5_eval[0]
print "+++=============+++"
print (float(exhaustive_eval[0]) / 160000.0) - 1000.0 # 3
print (float(gen_size_1_eval[0]) / 160000.0) - 1000.0 # 3
print (float(gen_size_2_eval[0]) / 160000.0) - 1000.0 # 5
print (float(gen_size_3_eval[0]) / 160000.0) - 1000.0 # 7
print (float(gen_size_4_eval[0]) / 160000.0) - 1000.0 # 7
print (float(gen_size_5_eval[0]) / 160000.0) - 1000.0 # 6


178275064
178333436
180167338
179798577
180005356
180320152
+++=============+++
114.21915
114.583975
126.0458625
123.74110625
125.033475
127.00095


In [15]:
others = []
others.append("IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(highestCardLE(ARG0, 9), 'check', 'raise'))")
others.append("IfThenElse(and_(highestCardLE(ARG0, 10), lowestCardLE(ARG0, 6)), 'fold', 'raise')")
others.append("IfThenElse(and_(highestCardLE(ARG0, 10), lowestCardLE(ARG0, 5)), 'check', 'raise')")
others.append("IfThenElse(highestCardGE(ARG0, 9), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', 'fold'))")
others.append("IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(highestCardGE(ARG0, 10), 'raise', IfThenElse(and_(lowestCardLE(ARG0, 5), and_(lowestCardGE(ARG0, 3), highestCardGE(ARG0, 8))), 'fold', IfThenElse(and_(isSameSuit(ARG0), lowestCardLE(ARG0, 7)), 'check', 'fold'))))")
others.append("IfThenElse(lowestCardGE(ARG0, 8), 'raise', IfThenElse(and_(and_(isSameSuit(ARG0), highestCardGE(ARG0, 2)), hasDoubles(ARG0)), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(and_(lowestCardGE(ARG0, 2), highestCardGE(ARG0, 10)), 'raise', 'fold'))))")
others.append("IfThenElse(highestCardGE(ARG0, 11), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(lowestCardGE(ARG0, 8), 'raise', 'check'))))")
others.append("IfThenElse(lowestCardGE(ARG0, 6), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(highestCardGE(ARG0, 10), 'raise', 'fold')))")
others.append("IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(highestCardGE(ARG0, 10), 'raise', IfThenElse(highestCardLE(ARG0, 4), 'fold', 'fold')))")
others.append("IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(lowestCardGE(ARG0, 9), 'raise', IfThenElse(highestCardLE(ARG0, 7), 'fold', IfThenElse(highestCardGE(ARG0, 10), 'raise', 'check'))))")
others.append("IfThenElse(lowestCardGE(ARG0, 11), 'raise', IfThenElse(lowestCardGE(ARG0, 6), 'raise', IfThenElse(and_(lowestCardLE(ARG0, 12), hasDoubles(ARG0)), 'raise', IfThenElse(highestCardGE(ARG0, 11), 'raise', IfThenElse(lowestCardGE(ARG0, 5), 'check', 'fold')))))")
others.append("IfThenElse(and_(and_(highestCardLE(ARG0, 7), isSameSuit(ARG0)), and_(lowestCardLE(ARG0, 5), hasDoubles(ARG0))), 'check', IfThenElse(lowestCardGE(ARG0, 7), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(highestCardGE(ARG0, 10), 'raise', IfThenElse(lowestCardGE(ARG0, 6), 'raise', 'fold')))))")
others.append("IfThenElse(and_(and_(highestCardGE(ARG0, 13), lowestCardGE(ARG0, 6)), and_(isSameSuit(ARG0), hasDoubles(ARG0))), 'check', IfThenElse(lowestCardGE(ARG0, 7), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(highestCardGE(ARG0, 11), 'raise', IfThenElse(lowestCardGE(ARG0, 6), 'raise', 'fold')))))")
others.append("IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(highestCardLE(ARG0, 9), 'check', 'raise'))")
others.append("IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(highestCardLE(ARG0, 9), 'fold', 'raise'))")
others.append("IfThenElse(and_(lowestCardGE(ARG0, 10), isSameSuit(ARG0)), 'raise', IfThenElse(and_(lowestCardLE(ARG0, 5), highestCardLE(ARG0, 11)), 'check', 'raise'))")

others_primitive_tree = []
for heuristic in others:
    others_primitive_tree.append(gp.PrimitiveTree.from_string(heuristic, pset))

evals = []

for i in range(0, len(others_primitive_tree)):
    evals.append(evaluate(others_primitive_tree[i], n=160000))

In [59]:
for h in others_primitive_tree:
    if not isTreeValid(h, all_hands):
        print h

IfThenElse(lowestCardGE(ARG0, 8), 'raise', IfThenElse(and_(and_(isSameSuit(ARG0), highestCardGE(ARG0, 2)), hasDoubles(ARG0)), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(and_(lowestCardGE(ARG0, 2), highestCardGE(ARG0, 10)), 'raise', 'fold'))))
IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(highestCardGE(ARG0, 10), 'raise', IfThenElse(highestCardLE(ARG0, 4), 'fold', 'fold')))
IfThenElse(and_(and_(highestCardLE(ARG0, 7), isSameSuit(ARG0)), and_(lowestCardLE(ARG0, 5), hasDoubles(ARG0))), 'check', IfThenElse(lowestCardGE(ARG0, 7), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(highestCardGE(ARG0, 10), 'raise', IfThenElse(lowestCardGE(ARG0, 6), 'raise', 'fold')))))
IfThenElse(and_(and_(highestCardGE(ARG0, 13), lowestCardGE(ARG0, 6)), and_(isSameSuit(ARG0), hasDoubles(ARG0))), 'check', IfThenElse(lowestCardGE(ARG0, 7), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(highestCardGE(ARG0, 11), 'raise', IfThenElse(lowestCardGE(ARG0, 6), 'raise', 'fold')))))


In [18]:
def findComplexity(ind):
    complexity = 0
    
    for expr in ind:
        if expr.name != "IfThenElse" and expr.name != "and_":
            if isinstance(expr, gp.Terminal):
                if expr.name == "raise" or expr.name == "fold" or expr.name == "check":
                    complexity += 1
            else:
                complexity += 1
    
    return complexity

In [16]:
print evals

[(176955134,), (177112744,), (176122478,), (176441785,), (177002251,), (176688032,), (177093905,), (177062958,), (177388458,), (176661862,), (177269351,), (176181698,), (176912375,), (177157263,), (177031339,), (176273812,)]


In [20]:
for i in range(0, len(others)):
    print str([(float(evals[i][0]) / 160000.0) - 1000.0, findComplexity(others_primitive_tree[i])]) + ","

[105.96958749999999, 5],
[106.9546499999999, 4],
[100.76548750000006, 4],
[102.76115625000011, 5],
[106.26406874999998, 12],
[104.3001999999999, 12],
[106.83690625000008, 9],
[106.64348749999999, 7],
[108.67786249999995, 7],
[104.1366375, 9],
[107.93344374999992, 12],
[101.13561249999998, 14],
[105.70234374999995, 14],
[107.2328937499999, 5],
[106.44586875000005, 5],
[101.71132499999999, 7],


In [17]:
178686756.0 / 160000.0 - 1000.0

116.79222499999992

In [36]:
def runSimulationForTrees(n):
    number_of_players = NUM_OF_PLAYERS
    total_player_pot = TOTAL_PLAYER_POT
    small_blind = BIG_BLIND / 2

    heuristics = [
        "IfThenElse(highestCardGE(ARG0, 9), 'raise', 'fold')",
        "IfThenElse(and_(highestCardLE(ARG0, 10), lowestCardLE(ARG0, 5)), 'check', 'raise')",
        "IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(highestCardLE(ARG0, 9), 'check', 'raise'))",
        "IfThenElse(highestCardGE(ARG0, 10), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(isSameSuit(ARG0), 'check', 'fold')))",
        "IfThenElse(lowestCardGE(ARG0, 6), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(highestCardGE(ARG0, 10), 'raise', 'fold')))",
        "IfThenElse(highestCardGE(ARG0, 11), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(isSameSuit(ARG0), 'fold', IfThenElse(lowestCardGE(ARG0, 8), 'raise', 'check'))))",
        "IfThenElse(lowestCardGE(ARG0, 8), 'raise', IfThenElse(and_(and_(isSameSuit(ARG0), highestCardGE(ARG0, 2)), hasDoubles(ARG0)), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(and_(lowestCardGE(ARG0, 2), highestCardGE(ARG0, 10)), 'raise', 'fold'))))",
        "IfThenElse(and_(and_(highestCardLE(ARG0, 7), isSameSuit(ARG0)), and_(lowestCardLE(ARG0, 5), hasDoubles(ARG0))), 'check', IfThenElse(lowestCardGE(ARG0, 7), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(highestCardGE(ARG0, 10), 'raise', IfThenElse(lowestCardGE(ARG0, 6), 'raise', 'fold')))))",
    ]
    
    agents = []
    for i in range(0, number_of_players):
        agents.append(HeuristicAI())
        agents[i].setHeuristic(toolbox.compile(gp.PrimitiveTree.from_string(heuristics[i], pset)))
    #agents.append(AlwaysRaiseAI())

    #agents[0].setHeuristic(toolbox.compile(individual))
    #agents[1].setHeuristic(toolbox.compile(individual))
    
    gh = GameHandler(number_of_players, agents, total_player_pot, small_blind)
    
    results = []
    agent_pot = []
    
    for i in range(0, n):
        result, money = gh.runGetAllPlayerPots()
        results.append(result)
        agent_pot.append(money)
    
    return results, agent_pot

In [46]:
results, agent_pot = runSimulationForTrees(160000)

In [41]:
def calculateStats(results, agent_pot):
    wins = {}
    money = {}
    for x in results:
        for num in x:
            if num not in wins:
                wins[num] = 1
                money[num] = 0
            else:
                wins[num] += 1

    for agent_money in agent_pot:
        for i in range(0, len(agent_money)):
            money[i] += agent_money[i]
    
    return wins, money

In [47]:
wins, money = calculateStats(results, agent_pot)
print "Wins: "
for x in wins:
    print str(x+1) + " : " + str(wins[x])
print "+++++==========+++++"
print "Money: "
for x in money:
    print str(x+1) + " : " + str(float(money[x]) / 160000.0 - 1000.0)


Wins: 
1 : 19531
2 : 25597
3 : 25665
4 : 20590
5 : 20011
6 : 22511
7 : 18817
8 : 20316
+++++==========+++++
Money: 
1 : 13.186375
2 : -42.78880625
3 : -37.6318875
4 : 26.89855625
5 : 16.3258375
6 : -33.41375625
7 : 31.96525
8 : 25.43853125


In [67]:
def outputReadableHeuristic(heuristic_tree):
    result = ""
    default = "IF "
    and_count = 0
    for expr in heuristic_tree:
        if expr.name == "IfThenElse":
            result += default
            default = "ELSE IF "
            
        elif "LE" == expr.name[-2:]:
            result += expr.name[:-2] + " <= "
        elif "GE" == expr.name[-2:]:
            result += expr.name[:-2] + " <= "
        elif isinstance(expr, gp.Terminal):
            if "ARG" in expr.name:
                continue
            if expr.ret == str:
                if expr == heuristic_tree[-1]:
                    result += "ELSE " + expr.name.upper()
                else:
                    result += "THEN " + expr.name.upper() + "\n"
            else:
                result += expr.name + " "
                if and_count > 0:
                    result += " AND "
                    and_count -= 1
        elif expr.name != "and_":
            result += expr.name + " "
            if and_count > 0:
                result += " AND "
                and_count -= 1
        else:
            and_count += 1
    
    return result

In [127]:
temp = evaluate(others_primitive_tree[0], n=160000)

In [128]:
(float(temp[0]) / 160000.0) - 1000.0

106.53813749999995